#### Q1. Use the API to get all available data for the GDP per capita, PPP (constant 2021 international $) indicator. Hint: this indicator has code "NY.GDP.PCAP.PP.KD". Adjust the query parameters so that you can retrieve all available rows. Convert the results to a DataFrame.

In [1]:
import requests
import pandas as pd

In [3]:
endpoint = 'https://api.worldbank.org/v2/country/all/indicator/NY.GDP.PCAP.PP.KD?per_page=17024'
gdp_df = pd.read_xml(endpoint)

#### Q2. Now, use the API to get all available data for Life expectancy at birth, total (years). This indicator has code "SP.DYN.LE00.IN". Again, convert the results to a DataFrame.

In [20]:
endpoint2 = 'https://api.worldbank.org/v2/country/all/indicator/SP.DYN.LE00.IN?per_page=17024'
le_df = pd.read_xml(endpoint2)
le_df.head()

#### Q3. Merge the two results DataFrames together. You may want to rename or drop columns prior to merging.

In [13]:
gdp_df = (
    gdp_df
    .drop(columns=['indicator', 'countryiso3code', 'unit', 'obs_status', 'decimal'])
    .rename(columns={'country': 'Country', 'date': 'Year', 'value': 'GDP_per_capita'})
)
gdp_df.head()

,Country,Year,GDP_per_capita
0,Africa Eastern and Southern,2023,4047.007031
1,Africa Eastern and Southern,2022,4038.638689
2,Africa Eastern and Southern,2021,3994.171654
3,Africa Eastern and Southern,2020,3919.499230
4,Africa Eastern and Southern,2019,4130.057222


In [15]:
le_df = (
    le_df
    .drop(columns=['indicator', 'countryiso3code', 'unit', 'obs_status', 'decimal'])
    .rename(columns={'country': 'Country', 'date': 'Year', 'value': 'Life_expectancy'})
)
le_df.head()

,Country,Year,Life_expectancy
0,Africa Eastern and Southern,2023,NaN
1,Africa Eastern and Southern,2022,62.899031
2,Africa Eastern and Southern,2021,62.454590
3,Africa Eastern and Southern,2020,63.313860
4,Africa Eastern and Southern,2019,63.755678


In [17]:
gdp_le_df = pd.merge(gdp_df, le_df, on = ['Country', 'Year'], how = 'inner', validate='one_to_one')
gdp_le_df

,Country,Year,GDP_per_capita,Life_expectancy
0,Africa Eastern and Southern,2023,4047.007031,NaN
1,Africa Eastern and Southern,2022,4038.638689,62.899031
2,Africa Eastern and Southern,2021,3994.171654,62.454590
3,Africa Eastern and Southern,2020,3919.499230,63.313860
4,Africa Eastern and Southern,2019,4130.057222,63.755678
...,...,...,...,...
17019,Zimbabwe,1964,NaN,54.994000
17020,Zimbabwe,1963,NaN,54.549000
17021,Zimbabwe,1962,NaN,54.071000
17022,Zimbabwe,1961,NaN,53.619000


#### Q4. You can also get more information about the available countries (region, capital city, income level classification, etc.) by using the Country API. Use this API to pull in all available data. Merge this with your other datasets. Use this to now remove the rows that correspond to regions and not countries.

In [22]:
country_df = pd.read_xml('https://api.worldbank.org/v2/country?per_page=296')
country_df

,id,iso2Code,name,region,adminregion,incomeLevel,lendingType,capitalCity,longitude,latitude
0,ABW,AW,Aruba,Latin America & Caribbean,None,High income,Not classified,Oranjestad,-70.0167,12.5167
1,AFE,ZH,Africa Eastern and Southern,Aggregates,None,Aggregates,Aggregates,None,NaN,NaN
2,AFG,AF,Afghanistan,South Asia,South Asia,Low income,IDA,Kabul,69.1761,34.5228
3,AFR,A9,Africa,Aggregates,None,Aggregates,Aggregates,None,NaN,NaN
4,AFW,ZI,Africa Western and Central,Aggregates,None,Aggregates,Aggregates,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
291,XZN,A5,Sub-Saharan Africa excluding South Africa and ...,Aggregates,None,Aggregates,Aggregates,None,NaN,NaN
292,YEM,YE,"Yemen, Rep.",Middle East & North Africa,Middle East & North Africa (excluding high inc...,Low income,IDA,Sana'a,44.2075,15.3520
293,ZAF,ZA,South Africa,Sub-Saharan Africa,Sub-Saharan Africa (excluding high income),Upper middle income,IBRD,Pretoria,28.1871,-25.7460
294,ZMB,ZM,Zambia,Sub-Saharan Africa,Sub-Saharan Africa (excluding high income),Lower middle income,IDA,Lusaka,28.2937,-15.3982


In [26]:
country_df = (
    country_df
    .drop(columns=['id', 'iso2Code', 'adminregion', 'lendingType', 'longitude', 'latitude'])
    .rename(columns={'name': 'Country', 'region': 'Region', 'incomeLevel': 'Income_level', 'capitalCity': 'Capital'})
)
country_df.head()

,Country,Region,Income_level,Capital
0,Aruba,Latin America & Caribbean,High income,Oranjestad
1,Africa Eastern and Southern,Aggregates,Aggregates,None
2,Afghanistan,South Asia,Low income,Kabul
3,Africa,Aggregates,Aggregates,None
4,Africa Western and Central,Aggregates,Aggregates,None


In [30]:
gdp_le_df_clean = pd.merge(gdp_le_df, country_df, on = ['Country'], how = 'inner', validate='many_to_one')
gdp_le_df_clean

,Country,Year,GDP_per_capita,Life_expectancy,Region,Income_level,Capital
0,Africa Eastern and Southern,2023,4047.007031,NaN,Aggregates,Aggregates,None
1,Africa Eastern and Southern,2022,4038.638689,62.899031,Aggregates,Aggregates,None
2,Africa Eastern and Southern,2021,3994.171654,62.454590,Aggregates,Aggregates,None
3,Africa Eastern and Southern,2020,3919.499230,63.313860,Aggregates,Aggregates,None
4,Africa Eastern and Southern,2019,4130.057222,63.755678,Aggregates,Aggregates,None
...,...,...,...,...,...,...,...
16891,Zimbabwe,1964,NaN,54.994000,Sub-Saharan Africa,Lower middle income,Harare
16892,Zimbabwe,1963,NaN,54.549000,Sub-Saharan Africa,Lower middle income,Harare
16893,Zimbabwe,1962,NaN,54.071000,Sub-Saharan Africa,Lower middle income,Harare
16894,Zimbabwe,1961,NaN,53.619000,Sub-Saharan Africa,Lower middle income,Harare


In [34]:
gdp_le_df_clean = gdp_le_df_clean[gdp_le_df_clean['Region'] != 'Aggregates']

In [42]:
gdp_le_df_clean.sample(10)

,Country,Year,GDP_per_capita,Life_expectancy,Region,Income_level,Capital
15309,Thailand,2010,16805.485336,76.131000,East Asia & Pacific,Upper middle income,Bangkok
3693,Australia,1978,NaN,73.674146,East Asia & Pacific,High income,Canberra
4680,Brazil,2015,18042.809377,74.332000,Latin America & Caribbean,Upper middle income,Brasilia
12331,North Macedonia,1980,NaN,69.247122,Europe & Central Asia,Upper middle income,Skopje
15210,Tajikistan,1981,NaN,59.456000,Europe & Central Asia,Lower middle income,Dushanbe
5983,Costa Rica,1992,11633.505325,76.627000,Latin America & Caribbean,Upper middle income,San Jose
13889,Sierra Leone,2022,1643.991981,60.411000,Sub-Saharan Africa,Low income,Freetown
16411,Vanuatu,1996,3159.996426,68.630000,East Asia & Pacific,Lower middle income,Port-Vila
10783,Maldives,1992,9050.510210,65.007000,South Asia,Upper middle income,Male
4251,Belize,1996,9765.187412,69.806000,Latin America & Caribbean,Upper middle income,Belmopan
